In [1]:
import numpy as np
import pandas as pd

In [2]:
# the number of segments; when we deploy repeater
N = np.array([1,2,4,8,16,32,64,128])

# metal parameter
metal_r = 120 / 1e-6
metal_c = (0.2) * 1e-15 / 1e-6
WL_length = 256 * 1e-6

# cell parameter
cell_num = 256
cell_c = 1e-15
cell_C = cell_c * cell_num

###################################################################
### TR/Logic Gates parameters: based on P/N ratio 7:3 (2.57:1.1)###
###################################################################

# TR parameter
# gate cap
tau = (9/2) * 1e-12 #measured FO1 delay : tau(1+pinv)
TR_C = (1.5/1.1) * 1e-15 # measured
#tau = (10/3) * TR_C * TR_R
TR_R = tau / ((10/3)*TR_C)




# Logic Gates
g_inv, p_inv = 1, 309/135

g_nand2, g_nand3, g_nand6 = 1.3 , 1.6, 2.5
p_nand2, p_nand3, p_nand6 = p_inv*2, p_inv*3, p_inv*6

g_nor2, g_nor3 = 1.7, 2.4
p_nor2, p_nor3 = p_inv*2, p_inv*3

# Decoder Logic
G_nand6_inv = g_nand6 * g_inv
P_nand6_inv = p_nand6 + p_inv
G_nand3_nor2 = g_nand3 * g_nor2
P_nand3_nor2 = p_nand3 + p_nor2
B = 32 # critical path: CLK signal

In [3]:
# delay of decoder
WL_R = metal_r * WL_length
WL_C = metal_c * WL_length

C_out = (WL_C + cell_C)
C_in = 5 * 1e-15 * 32 #initial input load 5fF × 32
H = C_out/C_in

G = np.array([G_nand6_inv, G_nand3_nor2])
F = np.outer(G,H)*B

F = pd.DataFrame(data = F, columns = ['value'], index=['NAND6-INV','NAND3-NOR2'])
stages = np.log(F)/np.log(4)
stages

,value
NAND6-INV,3.631517
NAND3-NOR2,3.692356


In [4]:
#with 2 stages
f_2 = F ** (1/2)
delay_decoder_stage2 = 2*f_2
delay_decoder_stage2.loc['NAND6-INV'] = delay_decoder_stage2.loc['NAND6-INV'] + P_nand6_inv
delay_decoder_stage2.loc['NAND3-NOR2'] = delay_decoder_stage2.loc['NAND3-NOR2'] + P_nand3_nor2
delay_decoder_stage2 = delay_decoder_stage2 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage2','NAND3-NOR2':'NAND3-NOR2 Stage2'}
delay_decoder_stage2.rename(index = new_row_names, inplace = True)
delay_decoder_stage2

,value
NAND6-INV Stage2,1.836419e-10
NAND3-NOR2 Stage2,1.678463e-10


In [5]:
#with 4 stages
f_4 = F ** (1/4)
delay_decoder_stage4 = 4*f_4
delay_decoder_stage4.loc['NAND6-INV'] = delay_decoder_stage4.loc['NAND6-INV'] + P_nand6_inv + 2 * p_inv
delay_decoder_stage4.loc['NAND3-NOR2'] = delay_decoder_stage4.loc['NAND3-NOR2'] + P_nand3_nor2 + 2 * p_inv
delay_decoder_stage4 = delay_decoder_stage4 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage4','NAND3-NOR2':'NAND3-NOR2 Stage4'}
delay_decoder_stage4.rename(index = new_row_names, inplace = True)
delay_decoder_stage4

,value
NAND6-INV Stage4,1.560680e-10
NAND3-NOR2 Stage4,1.368184e-10


In [6]:
#with 6 stages
f_6 = F ** (1/6)
delay_decoder_stage6 = 6*f_6
delay_decoder_stage6.loc['NAND6-INV'] = delay_decoder_stage6.loc['NAND6-INV'] + P_nand6_inv + 4 * p_inv
delay_decoder_stage6.loc['NAND3-NOR2'] = delay_decoder_stage6.loc['NAND3-NOR2'] + P_nand3_nor2 + 4 * p_inv
delay_decoder_stage6 = delay_decoder_stage6 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage6','NAND3-NOR2':'NAND3-NOR2 Stage6'}
delay_decoder_stage6.rename(index = new_row_names, inplace = True)
delay_decoder_stage6

,value
NAND6-INV Stage6,1.757831e-10
NAND3-NOR2 Stage6,1.560676e-10


In [7]:
#with 8 stages
f_8 = F ** (1/8)
delay_decoder_stage8 = 8*f_8
delay_decoder_stage8.loc['NAND6-INV'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand6_inv + 6 * p_inv
delay_decoder_stage8.loc['NAND3-NOR2'] = delay_decoder_stage8.loc['NAND3-NOR2'] + P_nand3_nor2 + 6 * p_inv
delay_decoder_stage8 = delay_decoder_stage8 * tau
new_row_names = {'NAND6-INV':'NAND6-INV Stage8','NAND3-NOR2':'NAND3-NOR2 Stage8'}
delay_decoder_stage8.rename(index = new_row_names, inplace = True)
delay_decoder_stage8

,value
NAND6-INV Stage8,2.021622e-10
NAND3-NOR2 Stage8,1.815622e-10


In [8]:
delay_decoder = pd.concat([delay_decoder_stage2, delay_decoder_stage4, delay_decoder_stage6, delay_decoder_stage8])
delay_decoder

,value
NAND6-INV Stage2,1.836419e-10
NAND3-NOR2 Stage2,1.678463e-10
NAND6-INV Stage4,1.560680e-10
NAND3-NOR2 Stage4,1.368184e-10
NAND6-INV Stage6,1.757831e-10
NAND3-NOR2 Stage6,1.560676e-10
NAND6-INV Stage8,2.021622e-10
NAND3-NOR2 Stage8,1.815622e-10


In [9]:
# NMOS width of repeater drived from RC delay of N segments
W = ( N*(TR_R*(metal_c + 1e-15))/((10/3)*metal_r*TR_C) ) ** (1/2)

segment_delay = (TR_R/W) * ( (10*TR_C*p_inv*W/3) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * ( (10*TR_C*W/3) + cell_C/(2*N) + WL_C/(2*N) )
last_segment_delay = (TR_R/W) * ( (10*TR_C*p_inv*W/3) + cell_C/(2*N) + WL_C/(2*N) ) + (TR_R/W + WL_R/N) * (cell_C/(2*N) + WL_C/(2*N))
repeater_delay = 14.8 *1e-12 #measured FO1 delay
delay_WL = (N-1) * segment_delay + last_segment_delay   + (N-1) * repeater_delay
delay_WL

array([5.23367185e-09, 2.81561843e-09, 1.65733352e-09, 1.19400273e-09,
       1.19089594e-09, 1.62563040e-09, 2.67445234e-09, 4.79537128e-09])

In [10]:
delay_total = delay_decoder.copy()
for i in range(7):
  delay_total = pd.concat([delay_total, delay_decoder], axis=1)
delay_total = delay_total + delay_WL
delay_total.columns = [1,2,4,8,16,32,64,128]
delay_total

,1,2,4,8,16,32,64,128
NAND6-INV Stage2,5.417314e-09,2.999260e-09,1.840975e-09,1.377645e-09,1.374538e-09,1.809272e-09,2.858094e-09,4.979013e-09
NAND3-NOR2 Stage2,5.401518e-09,2.983465e-09,1.825180e-09,1.361849e-09,1.358742e-09,1.793477e-09,2.842299e-09,4.963218e-09
NAND6-INV Stage4,5.389740e-09,2.971686e-09,1.813402e-09,1.350071e-09,1.346964e-09,1.781698e-09,2.830520e-09,4.951439e-09
NAND3-NOR2 Stage4,5.370490e-09,2.952437e-09,1.794152e-09,1.330821e-09,1.327714e-09,1.762449e-09,2.811271e-09,4.932190e-09
NAND6-INV Stage6,5.409455e-09,2.991401e-09,1.833117e-09,1.369786e-09,1.366679e-09,1.801413e-09,2.850235e-09,4.971154e-09
NAND3-NOR2 Stage6,5.389739e-09,2.971686e-09,1.813401e-09,1.350070e-09,1.346964e-09,1.781698e-09,2.830520e-09,4.951439e-09
NAND6-INV Stage8,5.435834e-09,3.017781e-09,1.859496e-09,1.396165e-09,1.393058e-09,1.827793e-09,2.876615e-09,4.997533e-09
NAND3-NOR2 Stage8,5.415234e-09,2.997181e-09,1.838896e-09,1.375565e-09,1.372458e-09,1.807193e-09,2.856015e-09,4.976933e-09


In [11]:
delay_total.describe()

,1,2,4,8,16,32,64,128
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00,8.000000e+00
mean,5.403666e-09,2.985612e-09,1.827327e-09,1.363996e-09,1.360890e-09,1.795624e-09,2.844446e-09,4.965365e-09
std,2.028900e-11,2.028900e-11,2.028900e-11,2.028900e-11,2.028900e-11,2.028900e-11,2.028900e-11,2.028900e-11
min,5.370490e-09,2.952437e-09,1.794152e-09,1.330821e-09,1.327714e-09,1.762449e-09,2.811271e-09,4.932190e-09
25%,5.389740e-09,2.971686e-09,1.813401e-09,1.350071e-09,1.346964e-09,1.781698e-09,2.830520e-09,4.951439e-09
50%,5.405487e-09,2.987433e-09,1.829148e-09,1.365817e-09,1.362711e-09,1.797445e-09,2.846267e-09,4.967186e-09
75%,5.415754e-09,2.997701e-09,1.839416e-09,1.376085e-09,1.372978e-09,1.807713e-09,2.856534e-09,4.977453e-09
max,5.435834e-09,3.017781e-09,1.859496e-09,1.396165e-09,1.393058e-09,1.827793e-09,2.876615e-09,4.997533e-09


In [12]:
H

1.92

In [13]:
f_4

,value
NAND6-INV,3.520447
NAND3-NOR2,3.595465


In [14]:
W

array([0.60249632, 0.85205847, 1.20499264, 1.70411693, 2.40998528,
       3.40823387, 4.81997056, 6.81646773])

In [15]:
2.40998528 * 7 / 3

5.623298986666666